# Ejemplo Dplyr

Este es un ejemplo de cómo usar dplyr usando un ejemplo de una pupusería.


In [1]:
suppressMessages(library(dplyr))
suppressMessages(library(stringr))

tirar_pupusa <- function (tipo_pupusa="queso"){
    mensaje <- paste("Tirando pupusa",tipo_pupusa)
    #print(mensaje)
    estado_pupusa <- "pupusa hecha"
    return(estado_pupusa)
}

preparar_bebida <- function (tipo_bebida="agua"){
    mensaje <- paste("Sirviendo",tipo_bebida)
    #print(mensaje)
    return ("bebida servida")
}

#tirar_pupusa("revuelta")
#preparar_bebida("Suprema")

In [2]:
# Creamos el dataset de la orden
concepto <- c("pupusa revuelta",
              "pupusa queso con loroco", 
              "pupusa frijol con queso",
              "pupusa chipilin",
              "suprema",
              "horchata"
             )

cantidad <- c(1, 3, 3, 5, 2, 1)
orden <- data.frame(concepto, cantidad)
orden

concepto,cantidad
pupusa revuelta,1
pupusa queso con loroco,3
pupusa frijol con queso,3
pupusa chipilin,5
suprema,2
horchata,1


In [3]:
orden_entregada <-
    orden %>%
    filter(concepto != "pupusa chipilin") %>% #se filtran las pupusas de chipilin
    mutate(estado = if_else(
                concepto == "suprema" | concepto == "horchata", # condición
                preparar_bebida(concepto), # si es verdadero
                NA_character_ # si no es verdadero
            )
    ) %>%
    # se sirven las bebidas y se pone NA si no es suprema u horchata. Además se
    # agrega el campo estado, para saber si está hecho o no.
    mutate(estado = if_else(
                is.na(estado) & grepl("pupusa", concepto),
                tirar_pupusa(concepto),
                estado
            )
    ) %>%
    # Se preparan las pupusas y se deja el campo estado como estaba
    mutate(estado=replace(estado, str_detect(estado, "hecha|servida"), "hecho"))
    # Se reemplazan las palabras "hecha" y "servida" por "hecho"
orden_entregada

concepto,cantidad,estado
pupusa revuelta,1,hecho
pupusa queso con loroco,3,hecho
pupusa frijol con queso,3,hecho
suprema,2,hecho
horchata,1,hecho


In [4]:
# Creamos el dataset de los precios.
producto <- c("pupusa", "yuca frita", "horchata", "suprema")
precio <- c(1, 2, 1.5, 2.75)
lista_precios <- data.frame(producto, precio)
lista_precios

producto,precio
pupusa,1.00
yuca frita,2.00
horchata,1.50
suprema,2.75


In [5]:
orden_entregada %>%
    left_join(x=.,y=lista_precios, by=c("concepto" = "producto"))

Warning message:
"Column `concepto`/`producto` joining factors with different levels, coercing to character vector"

concepto,cantidad,estado,precio
pupusa revuelta,1,hecho,NA
pupusa queso con loroco,3,hecho,NA
pupusa frijol con queso,3,hecho,NA
suprema,2,hecho,2.75
horchata,1,hecho,1.50


In [6]:
#Arreglamos orden
orden_entregada<- 
    orden_entregada %>%
    mutate(concepto = str_replace(concepto, "pupusa.*", "pupusa"))
orden_entregada

concepto,cantidad,estado
pupusa,1,hecho
pupusa,3,hecho
pupusa,3,hecho
suprema,2,hecho
horchata,1,hecho


In [7]:
orden_entregada <-
    orden_entregada %>%
    left_join(x=.,y=lista_precios, by=c("concepto" = "producto"))
orden_entregada

Warning message:
"Column `concepto`/`producto` joining character vector and factor, coercing into character vector"

concepto,cantidad,estado,precio
pupusa,1,hecho,1.00
pupusa,3,hecho,1.00
pupusa,3,hecho,1.00
suprema,2,hecho,2.75
horchata,1,hecho,1.50


In [8]:
orden_entregada %>%
    mutate(total_unitario=cantidad*precio) %>%
    select(concepto, cantidad, total_unitario) %>%
    group_by(concepto) %>%
    summarise(total_unitario=sum(total_unitario))

concepto,total_unitario
horchata,1.5
pupusa,7.0
suprema,5.5
